Mass Spring Model
------------------

A mass sping model is a decent approximation for cloth or any thin shelled material.  The model is made up of a mesh of nodes interconnected with edges.  The nodes have a mass $m$ and the edges have a spring contant $k$, length $len$, and a natural resting length $len_{rest}$.  The force of gravity is included with, using the normal gravitational constant, $g=<0,0,-9.8>$.

Thus the force on each node is:
$$F = mg + \sum_{springs} k(len - len_{rest})$$

Then Newton's second equation provides the differential equations to update the position of the nodes, $x$.  A damepning factor $\lambda$ is included to approximate the effect of friction.  We use two coupled first order equations for each node:
$$\frac{dx}{dt} = v$$
$$\frac{dv}{dt} = \frac{F - \lambda v}{m}$$

Given this formulation of differential equations an initial velocity must be supplied, zero is the natural choice.

In order to solve the differential equations euler's method is used.

In [ ]:
# second order ODE solver using euler's method
#    f : y'' = f(y, y') where y'' is a 3D vector with dimensions <x,y,z>
#    n : number of slices to split the timespan into 
#   t0 : initial time
#   t1 : final time
#   x0 : initial position vector, is a 3D vector with dimensions <x,y,z>
#   v0 : initial value of velocity, is a 3D vector with dimensions <x,y,z>
# Returns value of y, y1 at xb. 
def Euler(f, t0, t1, x0, v0, n):
      h = (t1 - t0) / float(n)
      t = t0
      x = x0
      v = v0
      for i in range(n):
          v += h * f(y, y1)
          x += h * v
          #t += h
      return [x, v]

In [ ]:
# mesh data structure
x_lim = [0, 10];
y_lim = [0, 8];

n_nodes = 100;
z_height = 10;



